## 단순한 word2vec
- continuous bag-of-word(CBOW) 모델을 사용해 본다.

#### 3.2.1 CBOW 모델의 추론처리
- CBOW 모델은 맥락으로부터 target을 추측하는 용도의 신경망이다(target은 중앙 단어이고 그 주변 단어들이 '맥락'이다.)
- CBOW 모델이 가능한 정확하게 추론하도록 훈련시켜서 단어의 분산 표현을 얻어낼 것이다.
- CBOW 모델의 입력은 맥락이다. 맥락은 "you"와 "goodbye" 같은 단어들의 목록이다.
- 가장 먼저, 이 맥락을 원핫 표현으로 변환하여 CBOW 모델이 처리할 수 있도록 준비한다.

![image](https://github.com/choibigo/Study/assets/38881179/e4d39ac3-71a9-40a9-a646-d298d097ae8c)
- (각 층은 weight 연산 후 결과를 의미함)

- 입력층이 2개 있고 은닉층을 거쳐 출력층에 도달한다.
- 입력층이 2개인 이유는 2개의 맥락을 보고 추론을 하기 때문이다, 추론 결과는 각 단어가 있을 확률이 된다.
- 은닉층의 뉴련은 입력층의 완전연결계층에 의해 변환된 값이 되는데, 입력층이 여러 개 이면 전체를 '평균'하면 된다.
- 첫번째 입력층이 h1, 두 번째 입력층이 h2로 변환 되었다고 하면 은닉층 뉴런의 입력은 (h1 + h2)/2이다.
- 출력 층은 여기서는 총 7 개 인데 각각의 뉴런이 단어 1개에 대응 된다, 출력의 결과가 그 단어가될 확률로 해석할 수 있다.
- 최종적으로 Softmax 함수를 통해 '확률'을 얻을 수 있다.
- 입력층에서 은닉층으로의 변환은 완전연결계층(가중치는 Win)에 의해서 이뤄진다.
- 완전연결계층의 가중치는 Win은 7x3 행렬이며, 가중치가 바로 단어의 분산 표현 이다.

![image](https://github.com/choibigo/Study/assets/38881179/8b7a066a-8b1a-4375-abad-1a93070df2fd)

- 가중치 Win의 각 행에는 해당 단어의 분산 표현이 담겨 있다고 볼 수 있다.
- 학습을 진행할수록 맥락에서 출현하는 단어를 잘 추측하는 방향으로 이 분산(weight)가 갱신될 것이다.
- 맥락을 입력 받아 실제 단어를 예측하는 모델을 학습 시키는데 이때 wegiht는 입력보다 차원이 낮은 hidden layer로 가기위해 사용된다(차원 축소에 해당) 이때 더 낮은 차원으로 가도록 weight는 학습될 것이고 이 결과 weight 자체를 단어 분산 표현으로 사용할 수 있게 된다.
- 은닉 층의 차원이 입력 층의 차원보다 작은 것이 핵심 이다. 이를 이용하여 벡터의 차원을 축소할 수 있게 한다.
- 은닉층으로 가는 weight는 단어 예측에 필요한 정보를 '간결하게'만들 weight가 학습된다. 이결과 밀집벡터 표현을 얻을수 있게 된다.

![image](https://github.com/choibigo/Study/assets/38881179/7bee685a-4224-40c5-a0b4-94c9f08dcd3b)

- 계층관점에서 바라보기
- 가장 앞단에는 입력에 대해 weight와 내적하는 2개의 Matmul 계층이 있다.
- 이후 두 계층을 더한 뒤에 0.5를 곱하여 평균을 만든다.
- 최종적으로 평균이 은닉층의 입력으로 들어가게 되고 은닉층 weight와 Matmul 연산 이후 최종 Score가 출력된다.
- score는 모든 단어에 대해 그 단어일 확률을 담고 있다.

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Matmul

# 샘플 맥락 데이터
c0 = np.array([1, 0, 0, 0, 0, 0, 0])
c1 = np.array([0, 0, 1, 0, 0, 0, 0])

# 가충치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = Matmul(W_in)
in_layer1 = Matmul(W_in)
out_layer = Matmul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1) # 2개의 입력을 같은 Weight에 대해 행렬곱 이후 0.5를 곱해 평균낸다, weight를 공유함

s = out_layer.forward(h) # 위에서 만든 평균된 입력을 hidden layer에 통과 시킨후 output을 낸다.

print(s)


[ 2.38474678  0.72489415  1.10728004 -1.75258598  0.9539978  -0.33403008
  1.31766279]


- MatMul 계층은 1개만 생성한다, 이때 입력층 측의 MatMul 계층은 가중치 W_in을 공유한다.
- Matmul 게층들(in_layer0, in_layer1)의 forward() 메서드를 호출해 중간 데이터를 계산하고 출력층 측의 Matmul 계층(out_layer)을 통과시켜 각 단어의 점수를 구한다.

#### 3.2.2 CBOW 모델의 학습
- CBOW 모델은 출력층에서 각 단어의 점수를 출력했다.
- 이 점수에 소프트맥스 함수를 적용하면 '확률'을 구할 수 있다.
- 이 확률은 맥락이 주어졌을때 그 중앙에 어떤 단어가 출현하는지를 나타낸다.

![image](https://github.com/choibigo/Study/assets/38881179/a2fa7ddf-7bba-4bab-8788-ba091583e0de)

- CBOW 모델의 학습에서는 올바른 예측을 할 수 있도록 가중치를 조정하는 일을 합니다.
- 그 결과 가중치에 단어의 출편 패턴을 파악한 벡터가 학습된다.
- 지금까지의 실험에 의해 CBOW(이후 배울 skip-gram)로 얻을 수 있는 단어의 분산 표현은 단어의 의미 면에서나 문법 면에서 모두 우리의 직관에 부합하는 경우를 많이 볼 수 있었다.
- CBOW는 다중 분류 문제이기 때문에 softmax와 cross-entropy를 이용하면 된다.

![image](https://github.com/choibigo/S![image](https://github.com/choibigo/Study/assets/38881179/25958a2f-d42a-4f14-897b-e06e061833a6)tudy/assets/38881179/25958a2f-d42a-4f14-897b-e06e061833a6)

#### 3.2.3 word2vec의 가중치와 분산 표현
- word2vec에서 사용되는 신경망에는 두 가지 가중치가 있다, 완전연결계층의 가중치(Win)와 출력 측 완전연결계층의 가중치(Wout)이다.
- 그리고 입력 측 가중치 Win의 각 행이 각 단어의 분산 표현에 해당한다.
- Wout에도 단어의 의미가 인코딩된 벡터가 저장되어 있다고 할 시 있다. 다만, 출력 측 가중치는 열 방향으로 저장된다.

![image](https://github.com/choibigo/Study/assets/38881179/aba5067b-4586-435d-b68c-7cf63fce696f)

- weight를 선정하는 방법이 3가지 있을 수 있다.
    - 입력 측의 가중치만 사용하기
    - 출력 측의 가중치만 사용하기
    - 양쪽 가중치를 모두 사용학기
- word2vec(특히 skip-gram) '입력 측의 가중치만 사용하기'가 가장 대중적인 선택이다.
- 많은 연구에서 출력측 가중치는 버리고 입력 측 가중치만 사용하여 단어의 분산 표현으로 이용한다.